In [13]:
import torch 
from transformers import LlamaForSequenceClassification
from transformers import AutoTokenizer
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)

In [14]:
model_path = 'openlm-research/open_llama_3b' # https://huggingface.co/openlm-research/open_llama_3b
num_labels = 4 # 'Positive', 'Neutral', 'Negative', 'Irrelevant' (https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2)

model = LlamaForSequenceClassification.from_pretrained(model_path, num_labels=num_labels, torch_dtype=torch.float16, device_map='mps')
# Tokenize the input: Use the AutoTokenizer class from the transformers library to tokenize your input text. Set the tokenizer options according to your classification task.

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=False,
    trust_remote_code=True,
    padding_side="left",
    pad_token="<|endoftext|>"
)

model.resize_token_embeddings(len(tokenizer)) # https://github.com/huggingface/transformers/issues/1805

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at openlm-research/open_llama_3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(32001, 3200)

In [5]:
from transformers import AutoModelForSequenceClassification
#model2 = AutoModelForSequenceClassification.from_pretrained('openlm-research/open_llama_3b')

# Accede a la última capa principal
#last_layer = model.classifier
# Muestra la última capa principal
#print(last_layer)
last_hidden_layer = model.get_input_embeddings()
print(last_hidden_layer)

Embedding(32001, 3200)


In [3]:
model.eval()

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 3200)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (up_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear(in_features=8640, out_features=3200, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()

In [6]:
print(model)

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 3200)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (up_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear(in_features=8640, out_features=3200, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()

In [23]:
#Solo mostraba la info limitada que se da desde colab 
import sys 
#Copy code
# Supongamos que 'model' es tu modelo PyTorch
# Especifica el nombre del archivo en el que deseas guardar la representación textual del modelo
archivo_guardado = 'modelo.txt'

# Abre el archivo en modo escritura y redirige la salida estándar hacia el archivo
with open(archivo_guardado, 'w') as archivo:
    original_stdout = sys.stdout  # Guarda la salida estándar original
    sys.stdout = archivo  # Redirige la salida estándar al archivo
    print(model)  # Imprime el modelo (la salida se redirige al archivo)
    sys.stdout = original_stdout  # Restaura la salida estándar original

# El modelo ha sido impreso en el archivo especificado

In [15]:
total_layers = len(list(model.modules()))
print(total_layers)

370


In [8]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

model.embed_tokens.weight 	 torch.Size([32001, 3200])
model.layers.0.self_attn.q_proj.weight 	 torch.Size([3200, 3200])
model.layers.0.self_attn.k_proj.weight 	 torch.Size([3200, 3200])
model.layers.0.self_attn.v_proj.weight 	 torch.Size([3200, 3200])
model.layers.0.self_attn.o_proj.weight 	 torch.Size([3200, 3200])
model.layers.0.mlp.gate_proj.weight 	 torch.Size([8640, 3200])
model.layers.0.mlp.up_proj.weight 	 torch.Size([8640, 3200])
model.layers.0.mlp.down_proj.weight 	 torch.Size([3200, 8640])
model.layers.0.input_layernorm.weight 	 torch.Size([3200])
model.layers.0.post_attention_layernorm.weight 	 torch.Size([3200])
model.layers.1.self_attn.q_proj.weight 	 torch.Size([3200, 3200])
model.layers.1.self_attn.k_proj.weight 	 torch.Size([3200, 3200])
model.layers.1.self_attn.v_proj.weight 	 torch.Size([3200, 3200])
model.layers.1.self_attn.o_proj.weight 	 torch.Size([3200, 3200])
model.layers.1.mlp.gate_proj.weight 	 torch.Size([8640, 3200])
model.layers.1.mlp.up_proj.weight 	 torch

In [9]:
from torchsummary import summary
summary(model)


Layer (type:depth-idx)                        Param #
├─LlamaModel: 1-1                             --
|    └─Embedding: 2-1                         102,403,200
|    └─ModuleList: 2-2                        --
|    |    └─LlamaDecoderLayer: 3-1            123,910,400
|    |    └─LlamaDecoderLayer: 3-2            123,910,400
|    |    └─LlamaDecoderLayer: 3-3            123,910,400
|    |    └─LlamaDecoderLayer: 3-4            123,910,400
|    |    └─LlamaDecoderLayer: 3-5            123,910,400
|    |    └─LlamaDecoderLayer: 3-6            123,910,400
|    |    └─LlamaDecoderLayer: 3-7            123,910,400
|    |    └─LlamaDecoderLayer: 3-8            123,910,400
|    |    └─LlamaDecoderLayer: 3-9            123,910,400
|    |    └─LlamaDecoderLayer: 3-10           123,910,400
|    |    └─LlamaDecoderLayer: 3-11           123,910,400
|    |    └─LlamaDecoderLayer: 3-12           123,910,400
|    |    └─LlamaDecoderLayer: 3-13           123,910,400
|    |    └─LlamaDecoderLayer: 3-14 

Layer (type:depth-idx)                        Param #
├─LlamaModel: 1-1                             --
|    └─Embedding: 2-1                         102,403,200
|    └─ModuleList: 2-2                        --
|    |    └─LlamaDecoderLayer: 3-1            123,910,400
|    |    └─LlamaDecoderLayer: 3-2            123,910,400
|    |    └─LlamaDecoderLayer: 3-3            123,910,400
|    |    └─LlamaDecoderLayer: 3-4            123,910,400
|    |    └─LlamaDecoderLayer: 3-5            123,910,400
|    |    └─LlamaDecoderLayer: 3-6            123,910,400
|    |    └─LlamaDecoderLayer: 3-7            123,910,400
|    |    └─LlamaDecoderLayer: 3-8            123,910,400
|    |    └─LlamaDecoderLayer: 3-9            123,910,400
|    |    └─LlamaDecoderLayer: 3-10           123,910,400
|    |    └─LlamaDecoderLayer: 3-11           123,910,400
|    |    └─LlamaDecoderLayer: 3-12           123,910,400
|    |    └─LlamaDecoderLayer: 3-13           123,910,400
|    |    └─LlamaDecoderLayer: 3-14 

In [26]:
#pasaron mas de 1H y no finalizaba
import sys
from torchsummary import summary

# Supongamos que 'model' es tu modelo PyTorch
# Especifica el nombre del archivo en el que deseas guardar la información completa
archivo_guardado = 'summary_modelo.txt'

# Abre el archivo en modo escritura y redirige la salida estándar hacia el archivo
with open(archivo_guardado, 'w') as archivo:
    original_stdout = sys.stdout  # Guarda la salida estándar original
    sys.stdout = archivo  # Redirige la salida estándar al archivo
    summary(model, (input_size,))  # Genera y muestra la información del modelo
    sys.stdout = original_stdout  # Restaura la salida estándar original

# La información completa del modelo se guarda en el archivo especificado


## Lora

In [16]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"], # https://github.com/huggingface/peft/blob/632997d1fb776c3cf05d8c2537ac9a98a7ce9435/src/peft/utils/other.py#L202
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)

In [11]:
lora_model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(32001, 3200)
        (layers): ModuleList(
          (0-25): 26 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=3200, out_features=3200, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3200, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3200, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=3200, out_features=3200, bias=False)
              (v_proj): Linear(
   

In [17]:
total_layers_lora = len(list(lora_model.modules()))
print(total_layers_lora)

788
